In [1]:
%run ../app/models/model.py
from owlready2 import *
onto = get_ontology("../app/data/education.owl").load()

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
print("sử dụng SPARQL để tìm tất cả các học sinh có họ Lê")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?hs 
WHERE {
    ?hs a s:HocSinh.
    ?hs s:hoTen ?hoTen.
    FILTER (STRSTARTS(?hoTen, "Lê")).
}
"""))
for x in result: print(x[0].hoTen)

sử dụng SPARQL để tìm tất cả các học sinh có họ Lê
Lê Anh Thiên
Lê Thanh Bình


In [3]:
print("sử dụng SPARQL để tìm tất cả các học sinh sinh vào tháng 3")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?hs 
WHERE {
    ?hs a s:HocSinh.
    ?hs s:ngaySinh ?ngaySinh.
    FILTER (REGEX(?ngaySinh, ".*/03/.*", "i")).
}
"""))
for x in result: print(x[0].hoTen, x[0].ngaySinh)

sử dụng SPARQL để tìm tất cả các học sinh sinh vào tháng 3
Đoàn Thị Thúy 14/03/2000
Đoàn Thanh Bình 03/03/2000
Huỳnh Hồng Thảo 11/03/2000
Tôn Minh Ngọc 23/03/2000
Ngô Kim Tú 27/03/2000


In [4]:
print("sử dụng SPARQL để tìm tất cả các học sinh của lớp có tên là 10C03")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?hs 
WHERE {
    ?hs a s:HocSinh.
    ?hs s:hocLop ?lop.
    ?lop s:ten "10C03".
}
"""))
for x in result: print(x[0].hoTen, x[0].hocLop.ten)

sử dụng SPARQL để tìm tất cả các học sinh của lớp có tên là 10C03
Mai Thu Việt 10C03
Nguyễn Đình Minh 10C03
Đào Hồng Thiên 10C03
Phùng Bảo Tường 10C03
Lý Anh Nam 10C03


In [5]:
print("sử dụng SPARQL để tìm tất cả các học sinh của lớp có uri là #lophoc1")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?hs 
WHERE {
    ?hs a s:HocSinh.
    ?hs s:hocLop ?lop.
    FILTER(STRENDS(STR(?lop), "#lophoc1")).
}
"""))
for x in result: print(x[0].hoTen, x[0].hocLop.ten)

sử dụng SPARQL để tìm tất cả các học sinh của lớp có uri là #lophoc1
Bùi Minh Cường 10C01
Lâm Quý Minh 10C01
Bành Kim Vân 10C01
Hồ Hoài Hùng 10C01
Đoàn Thị Thúy 10C01


In [6]:
print("sử dụng SPARQL để tìm tất cả các giáo viên chủ nhiệm của từng lớp, sắp xếp theo tên lớp")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?gv
WHERE {
    ?gv a s:GiaoVien.
    ?gv s:lopChuNhiem ?lop.
    ?lop s:ten ?tenLop.
    FILTER(BOUND(?lop)).
}
ORDER BY ?tenLop
"""))
for x in result: print(' - '.join([x[0].hoTen, x[0].dayMon[0].ten, x[0].lopChuNhiem.ten]))

sử dụng SPARQL để tìm tất cả các giáo viên chủ nhiệm của từng lớp, sắp xếp theo tên lớp
Quách Thanh Hiền - Toán học - 10C01
Lê Hoài Quân - GDCD - 10C02
Tăng Hữu Ngọc - GDCD - 10C03
Nguyễn Minh Khánh - Địa lý - 11B01
Đinh Bảo Nhật - Hóa học - 11B02
Phùng Đình Thiện - Toán học - 11B03
Lâm Hữu Phương - Hóa học - 12A01
Bành Mai Linh - Lịch sử - 12A02
Huỳnh Hoài Long - Công nghệ - 12A03


In [7]:
print("sử dụng SPARQL để tìm tất cả các giáo viên chủ nhiệm của từng lớp (cách 2)")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?lop 
WHERE {
    ?lop a s:LopHoc.
}
"""))
for x in result: print(' - '.join([x[0].ten, x[0].giaoVienChuNhiem.hoTen]))

sử dụng SPARQL để tìm tất cả các giáo viên chủ nhiệm của từng lớp (cách 2)
10C01 - Quách Thanh Hiền
10C02 - Lê Hoài Quân
10C03 - Tăng Hữu Ngọc
11B01 - Nguyễn Minh Khánh
11B02 - Đinh Bảo Nhật
11B03 - Phùng Đình Thiện
12A01 - Lâm Hữu Phương
12A02 - Bành Mai Linh
12A03 - Huỳnh Hoài Long


In [8]:
print("sử dụng SPARQL để tìm tất cả các phụ huynh có 2 con đang học cùng trường")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?ph (COUNT(?hs) AS ?soCon)
WHERE {
    ?ph a s:PhuHuynh.
    ?ph s:con ?hs.
}
GROUP BY ?ph
HAVING (COUNT(?hs) = 2)
"""))
lst = list(map(lambda ph: [ph[0].hoTen, ph[0].con[0].hoTen, ph[0].con[1].hoTen], result))# Lấy họ tên phụ huynh và các con
lst.sort(key = lambda x:x[2]) # Sắp xếp theo tên con thứ 2
new_lst = [[a[0], b[0], a[1], a[2]] for a, b in zip(lst[::2], lst[1::2])] # Ghép lại theo cặp
for x in new_lst: print(f"({x[0]} + {x[1]}) = Con 1: {x[2]} - Con 2: {x[3]}")

sử dụng SPARQL để tìm tất cả các phụ huynh có 2 con đang học cùng trường
(Thái Ngọc Khánh + Thái Thanh Việt) = Con 1: Quách Thị Vinh - Con 2: Dương Mai Thúy
(Đoàn Mai Xuân + Huỳnh Văn Thiên) = Con 1: Lê Anh Thiên - Con 2: Hoàng Bảo Tú
(Vũ Quý Hải + Đinh Bảo Thuận) = Con 1: Lê Thanh Bình - Con 2: Hoàng Hoài Long
(Tô Anh Dũng + Trương Bảo Dũng) = Con 1: Quách Kim Vinh - Con 2: Hoàng Hữu Hiền
(Bành Đình Hải + Đoàn Hoài Vinh) = Con 1: Hồ Mai Tường - Con 2: Huỳnh Hồng Thảo
(Ngô Ngọc Tùng + Tạ Hồng Vinh) = Con 1: Bành Kim Vân - Con 2: Hồ Hoài Hùng
(Tôn Thu Dũng + Phạm Văn Hùng) = Con 1: Đào Hữu Nam - Con 2: Lương Mai Quân
(Lê Bảo Cường + Thái Anh Thuận) = Con 1: Lâm Quý Minh - Con 2: Mai Thu Việt
(Đào Hồng Minh + Trương Bảo Sơn) = Con 1: Trần Hoài Sơn - Con 2: Nguyễn Đình Minh
(Mai Ngọc Long + Tô Thu Quỳnh) = Con 1: Đinh Quý Nam - Con 2: Ngô Kim Tú
(Phùng Đình Thiện + Ngô Hồng Sơn) = Con 1: Đoàn Đình Nhật - Con 2: Phùng Bảo Tường
(Tăng Hữu Việt + Tô Anh Ngọc) = Con 1: Trương Thu Tùng - Con 2

In [9]:
print("sử dụng SPARQL để liệt kê các Giáo viên dạy môn Toán theo thứ tự tên")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?gv ?ten
WHERE {
    ?gv a s:GiaoVien.
    ?gv s:dayMon ?mon.
    ?mon s:ten "Toán học".
    ?gv s:hoTen ?hoTen.
    BIND(strafter(strafter(?hoTen, " "), " ") AS ?ten) .
}
ORDER BY ?ten
"""))
for x in result: print(x[0].hoTen)


sử dụng SPARQL để liệt kê các Giáo viên dạy môn Toán theo thứ tự tên
Tôn Hồng Cường
Quách Thanh Hiền
Phùng Đình Thiện
Phan Hồng Thuận


In [12]:
print("sử dụng SPARQL để liệt kê các Giáo viên dạy lớp 10C03 theo thứ tự tên môn học")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?day
WHERE {
    ?day a s:GiangDay.
    ?day s:lopHoc ?lop.
    ?lop s:ten "10C03".
    ?day s:monHoc ?mon.
    ?mon s:ten ?tenMon.
}
ORDER BY ?tenMon
"""))
for x in result: print(x[0].giaoVien.hoTen, x[0].monHoc.ten, x[0].lopHoc.ten)

sử dụng SPARQL để liệt kê các Giáo viên dạy lớp 10C03 theo thứ tự tên môn học
Huỳnh Hoài Long Công nghệ 10C03
Nguyễn Minh Vân GDCD 10C03
Lâm Hữu Phương Hóa học 10C03
Phạm Mai Tường Lịch sử 10C03
Tôn Hồng Cường Ngữ văn 10C03
Bùi Văn Khánh Sinh học 10C03
Phan Thanh Vân Tiếng Anh 10C03
Phùng Đình Thiện Toán học 10C03
Đoàn Thị Phương Địa lý 10C03


In [18]:
print("sử dụng SPARQL để tìm tất cả các gv dạy 2 môn")
result = list(default_world.sparql("""
PREFIX s: <http://hc.com/school#>
SELECT ?gv (COUNT(?day) AS ?soMon)
WHERE {
    ?gv a s:GiaoVien.
    ?gv s:dayMon ?day.
}
GROUP BY ?gv
HAVING (COUNT(?day) > 1)
"""))
for x in result: print(x[0].hoTen, x[1], [mon.ten for mon in x[0].dayMon])

sử dụng SPARQL để tìm tất cả các gv dạy 2 môn
Tôn Hồng Cường 2 ['Toán học', 'Ngữ văn']
Võ Anh Hoàng 2 ['Hóa học', 'Lịch sử']
Bùi Văn Khánh 2 ['Lịch sử', 'Sinh học']
Phan Thanh Vân 2 ['Địa lý', 'Tiếng Anh']
Phạm Hoài Tú 2 ['Tiếng Anh', 'Công nghệ']
